In [1]:
from datetime import date, datetime, timedelta
import ast
import os
import time
from datetime import date, datetime

import geopandas as gpd
import numpy as np
import pandas as pd
from dotenv import load_dotenv
from shapely.geometry import Point
from sqlalchemy import create_engine
import time

from mods import gmap

load_dotenv()

True

In [2]:
# E_取得gml檔案，並轉換成gdf物件
folder = r"C:\Users\add41\Documents\Data_Engineer\Project\example_data"
file = "COUNTY_MOI_1140318.gml"
target = os.path.join(folder, file)

gdf = gpd.read_file(target)

# 列出目標縣市（台北市被包含在新北市中），並設定英文代號
today = date.today().strftime("%Y/%m/%d")
target_city = ["臺北市"]
city_eng = ["TPEI"]
target_index = []
city_dict = {
    "臺北市":"TPEI"
}

# 將目標縣市的索引加入列表中
for city in target_city:
    target_index.append(gdf[gdf["名稱"] == city].index)

target_index

[Index([7], dtype='int64')]

In [3]:
## E_透過gmap API遍尋六都資料

# 定義關鍵字列表和對應的英文名
keyword_list = ["寵物美容", "寵物餐廳", "寵物用品"]
keyword_dict = {
    "寵物美容":"salon",
    "寵物餐廳":"restaurant",
    "寵物用品":"supplies"
}

# 根據存取的索引列表去取出每一個縣市的多邊形資料
for keyword in keyword_list:
    metadate = []
    for idx in target_index:
        name = city_dict[gdf.loc[idx]["名稱"].values[0]]
        city_geo_data = gdf.loc[idx].geometry.values[0]

        # 取得四極點
        min_x, min_y, max_x, max_y = city_geo_data.bounds

        # 設定搜尋半徑及步長
        radius_m = 2000
        step_m = 2000

        # 將距離換算成經緯度
        lat_step = step_m / 111000
        lon_step = step_m / (111000 * np.cos(np.radians((min_y + max_y) / 2)))

        # 產生在範圍內的所有座標點
        lat_point = np.arange(min_y, max_y, lat_step)
        lon_point = np.arange(min_x, max_x, lon_step)

        print(f"開始搜尋{name}的{keyword}資料")
        print(f"lat個數：{len(lat_point)}")
        print(f"lon個數：{len(lon_point)}")

        # 將所有取得的經緯度組合成所有座標點，並形成一個list
        points = []

        for lat_p in lat_point:
            for lon_p in lon_point:
                loc = (lon_p, lat_p)
                points.append(loc)

        print(f"座標點個數：{len(points)}")

        # 開始進行gmap資料爬取
        data = []
        key = os.environ.get("GMAP_KEY6")
        count = 1

        for loc in points:
            loc_p = Point(loc[0], loc[1])

            if city_geo_data.contains(loc_p):
                lat = loc[1]
                lon = loc[0]
                result_list = gmap.gmap_nearby_search(key=key, lat=lat, lon=lon, radius=radius_m, keyword=keyword)
                for i in result_list:
                    data.append(i)

                df = pd.DataFrame(data=data)
                df["update_time"] = datetime.now().strftime("%Y/%m/%d %H:%M:%S")

                save = r"C:\Users\add41\Documents\Data_Engineer\Project\example_data\gmap_full_search"
                file_name = f"gmap_{keyword_dict[keyword]}_{name}_TPEI.csv"
                save_path = os.path.join(save, file_name)
                df.to_csv(save_path, index=False, encoding="utf-8-sig")

                print(f"完成{name}的{keyword}的第{count}/{len(points)}個座標點（{round(float(loc[1]), 7)}, {round(float(loc[0]), 7)}）的搜尋，共有{len(result_list)}筆店家資料")
                count += 1
                time.sleep(1)

        city_metadata = {
            "city":name,
            "search_radius":radius_m,
            "step":step_m,
            "coord_count":count,
            "data_count":len(data),
            "type":keyword_dict[keyword],
            "update_date":today
        }
        metadate.append(city_metadata)
        print(f"完成{name}的{keyword}資料搜尋，總共搜尋{count}個座標點")

        # 將結果轉換成df並建立更新時間欄位
        df = pd.DataFrame(data=data)
        df["update_time"] = datetime.now().strftime("%Y/%m/%d %H:%M:%S")

        # 將檔案儲存為csv
        save = r"C:\Users\add41\Documents\Data_Engineer\Project\example_data\gmap_full_search"
        file_name = f"gmap_{keyword_dict[keyword]}_{name}_TPEI.csv"
        save_path = os.path.join(save, file_name)
        df.to_csv(save_path, index=False, encoding="utf-8-sig")

        print(f"已完成{name}資料儲存")
        time.sleep(3)

開始搜尋TPEI的寵物美容資料
lat個數：14
lon個數：11
座標點個數：154
正在取得第2頁資訊...
完成TPEI的寵物美容的第1/154個座標點（24.9785182, 121.5566214）的搜尋，共有37筆店家資料
完成TPEI的寵物美容的第2/154個座標點（24.9785182, 121.576516）的搜尋，共有9筆店家資料
完成TPEI的寵物美容的第3/154個座標點（24.9785182, 121.5964105）的搜尋，共有0筆店家資料
正在取得第2頁資訊...
正在取得第3頁資訊...
完成TPEI的寵物美容的第4/154個座標點（24.9965362, 121.5367269）的搜尋，共有60筆店家資料
正在取得第2頁資訊...
完成TPEI的寵物美容的第5/154個座標點（24.9965362, 121.5566214）的搜尋，共有38筆店家資料
完成TPEI的寵物美容的第6/154個座標點（24.9965362, 121.576516）的搜尋，共有10筆店家資料
完成TPEI的寵物美容的第7/154個座標點（24.9965362, 121.5964105）的搜尋，共有3筆店家資料
正在取得第2頁資訊...
正在取得第3頁資訊...
完成TPEI的寵物美容的第8/154個座標點（25.0145542, 121.4969379）的搜尋，共有60筆店家資料
正在取得第2頁資訊...
正在取得第3頁資訊...
完成TPEI的寵物美容的第9/154個座標點（25.0145542, 121.5367269）的搜尋，共有60筆店家資料
正在取得第2頁資訊...
完成TPEI的寵物美容的第10/154個座標點（25.0145542, 121.5566214）的搜尋，共有33筆店家資料
完成TPEI的寵物美容的第11/154個座標點（25.0145542, 121.576516）的搜尋，共有17筆店家資料
完成TPEI的寵物美容的第12/154個座標點（25.0145542, 121.5964105）的搜尋，共有4筆店家資料
正在取得第2頁資訊...
完成TPEI的寵物美容的第13/154個座標點（25.0325723, 121.4969379）的搜尋，共有32筆店家資料
正在取得第2頁資訊...
完成TPEI的寵物美容的第14/154個座標點

In [4]:
def in_boundary(city_geo_data, lat, lon):
    loc_p = Point(lon, lat)
    return city_geo_data.contains(loc_p)

In [5]:
folder = r"C:\Users\add41\Documents\Data_Engineer\Project\example_data"
file = "COUNTY_MOI_1140318.gml"
geo_path = os.path.join(folder, file)
gdf = gpd.read_file(geo_path)

bound_dict = {row["名稱"]:row.geometry for _, row in gdf.iterrows()}

In [6]:
# 設定資料夾、城市和類別列表
folder = r"C:\Users\add41\Documents\Data_Engineer\Project\example_data\gmap_full_search"
city_list = ["TPEI"]
type_list = ["salon", "restaurant", "supplies"]
city_dict = {
    "TPEI":"臺北市"
}

# 先按照類別迴圈
for type_ in type_list:
    columns = ['name', 'place_id', 'buss_status', "geometry", 'update_time']
    df_main = pd.DataFrame(columns=columns)

    # 依序將城市檔案讀入
    for city in city_list:
        file = f"gmap_{type_}_{city}_TPEI.csv"
        path = os.path.join(folder, file)
        df_city = pd.read_csv(path)

        # 只保留正常營業的地標
        mask1 = df_city["buss_status"] == "OPERATIONAL"
        df_city = df_city[mask1]

        # 根據place id去除重複
        df_city = df_city.drop_duplicates(subset=["place_id"], keep="first")

        # 去除無經緯度的資料
        df_city.dropna(subset="geometry", inplace=True)
        print(f"完成{city}資料處理，共{len(df_city)}筆資料")

        # 檢查店家是否在六都邊界範圍內
        city_ch_name = city_dict[city]
        city_geo_data = bound_dict[city_ch_name]

        boundary_list = []
        df_city["geometry"] = df_city["geometry"].apply(ast.literal_eval)
        for index, row in df_city.iterrows():
            lat = row["geometry"].get("lat", None)
            lon = row["geometry"].get("lng", None)
            if lat == None or lon == None:
                boundary_list.append(False)
            else:
                boundary_list.append(in_boundary(city_geo_data, lat, lon))

        df_city['in_boundary'] = boundary_list

        # 只保留在邊界範圍內的
        mask2 = (df_city['in_boundary'] == True)
        df_city = df_city[mask2]

        # 跟主表合併
        df_main = pd.concat([df_main, df_city], ignore_index=True)

    # 合併完成再去重複一次
    df_main = df_main.drop_duplicates(subset=["place_id"], keep="first")

    # 去除經緯度欄位
    df_main.drop(columns="geometry", axis=1, inplace=True)

    # 新增更新日期
    today = date.today().strftime('%Y/%m/%d')
    df_main["update_date"] = today

    # 存檔至地端
    combine_file = f"{type_}_TPEI_place_id.csv"
    combine_path = os.path.join(folder, combine_file)
    df_main.to_csv(combine_path, index=False, encoding="utf-8")
    print(f"已完成{type_}地端存檔")

完成TPEI資料處理，共503筆資料
已完成salon地端存檔
完成TPEI資料處理，共519筆資料
已完成restaurant地端存檔
完成TPEI資料處理，共360筆資料
已完成supplies地端存檔
